In [58]:
%%capture
%run 3-tfidf-gensim.ipynb

In [59]:
from gensim.models import CoherenceModel, LdaModel
# %pip install pyldavis==3.3.1
import pyLDAvis
import pyLDAvis.gensim

In [60]:
# Fit an LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=20,
                     alpha='auto', eta='auto', passes=10, random_state=1)

In [61]:
# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['Content'], dictionary=id2word, coherence='c_v')

try:
    coherence_lda = coherence_model_lda.get_coherence()
    print(f'Coherence Score: {coherence_lda}')
except RuntimeWarning as e:
    print(f'Warning: {e}')
except ValueError as e:
    print(f'Error computing coherence score: {e}')

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Coherence Score: nan


In [39]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

In [30]:
lda_model[corpus]

# it returns a list of lists, where each inner list contains tuples where the topic ID and the topic probability are contained
# Each inner list is associated to a specific document

# output example
''' [
  DOC1: [(0, 0.05), (1, 0.09), ...]
  DOC2: [(0, 0.1), ...]
  .
  .
  .  
]
'''

' [\n  DOC1: [(0, 0.05), (1, 0.09), ...]\n  DOC2: [(0, 0.1), ...]\n  .\n  .\n  .  \n]\n'

In [31]:
# this function returns the top words for each topic by highest frequency
lda_model.show_topic(0, topn=10)

[('health', 0.023669543),
 ('space', 0.021356385),
 ('key', 0.016416237),
 ('package', 0.012306949),
 ('coverage', 0.0120579675),
 ('propaganda', 0.011376902),
 ('care', 0.010223077),
 ('program', 0.010132736),
 ('system', 0.009944139),
 ('mission', 0.009716835)]

In [54]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df_clean):
    # Init output
    sent_topics_df = pd.DataFrame(columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        # we want to sort the tuples in the list 'row' by descending order of the second element, which is the topic probability
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic, i.e. tuple with the highest topic probability
                wp = ldamodel.show_topic(topic_num, topn = 5)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df.loc[len(sent_topics_df)] = [int(topic_num), round(prop_topic, 4), topic_keywords]
            else:
                break
    # Add original text to the end of the output
    sent_topics_df = pd.concat([sent_topics_df, texts], axis=1)
    return sent_topics_df

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df_clean)


In [55]:
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Content
0,18,0.4140,"government, time, law, state, war",atheist music atheism music fiction distributi...
1,9,0.6123,"god, life, evidence, way, religion",introduction file atheism distribution world o...
2,18,0.4513,"government, time, law, state, war",gospel organization theology evidence luke con...
3,9,0.4472,"god, life, evidence, way, religion",separation organization mantis become none fly...
4,2,0.3414,"science, computer, insurance, center, research",strom al ax fund boy distribution organization...
...,...,...,...,...
11309,9,0.6048,"god, life, evidence, way, religion",promise organization buffalo god peace
11310,9,0.6814,"god, life, evidence, way, religion",promise organization tourist bureau barney spe...
11311,9,0.6058,"god, life, evidence, way, religion",protestant organization tourist bureau concept...
11312,14,0.5679,"car, gas, fire, house, sport",cutter backing tape distribution world organiz...


In [46]:
df_topic_sents_keywords['Dominant_Topic'].unique()

array([18,  9,  2,  8,  1, 19,  6, 13,  0,  5, 10, 14, 16, 12,  4,  3, 17,
        7, 15, 11], dtype=int64)

In [47]:
df_topic_distribution = df_topic_sents_keywords.groupby(['Dominant_Topic', 'Topic_Keywords']).size().sort_values(ascending=False).reset_index(name='count')

df_topic_distribution

,Dominant_Topic,Topic_Keywords,count
0,18,"government, time, law, state, war",2962
1,9,"god, life, evidence, way, religion",1329
2,4,"window, time, file, program, space",1153
3,1,"card, sale, coverage, drive, video",730
4,8,"program, version, key, proposal, gateway",722
5,2,"science, computer, insurance, center, research",650
6,10,"bike, mouse, drive, power, advice",602
7,19,"player, mac, apple, team, chip",588
8,14,"car, gas, fire, house, sport",516
9,16,"game, baseball, ten, season, year",491
